In [1]:
# ============================================================
# CONFIG SECTION — EDIT ONLY THIS
# ============================================================

WORK_DIR = r"D:\work\Client\Morgan Stanley\Model Results"

RAW_DATA_PATH = r"D:\work\Client\Morgan Stanley\Model Results\MSZZMOMO_closePriceL_tPLUS2_w_ETF_Features_rawdata.csv"

MODEL_PKL_PATH = r"D:\work\Client\Morgan Stanley\Model Results\L_tPLUS2_GBM_18M_CORE_top50_production.pkl"

TOP_FEATURES_CSV_PATH = r"D:\work\Client\Morgan Stanley\Model Results\L_tPLUS2_GBM_18M_CORE_top50_features.csv"

OUTPUT_PRED_PATH = r"D:\work\Client\Morgan Stanley\Model Results\MSZZMOMO_GBM_18M_CORE_top50_latest_preds.csv"

DATE_COL = "DATE"

# Predict only on last N rows of the raw file
N_PREDICT_LAST_ROWS = 2

# Cutoff for 1/0 signal
PROBA_CUTOFF = 0.58

# ============================================================
# DO NOT EDIT ANYTHING BELOW THIS LINE
# ============================================================

import os
from pathlib import Path
import numpy as np
import pandas as pd
import joblib

# Set working directory
if WORK_DIR:
    os.chdir(WORK_DIR)
    print("CWD:", os.getcwd())


# ============================================================
# Technical Features (same as training)
# ============================================================

def add_technical_features(df, core_F_cols, etf_base_cols):
    if "Lagged_tPLUS2" in df.columns:
        df["Lagged_tPLUS2"] = pd.to_numeric(df["Lagged_tPLUS2"], errors="coerce")
        lr = df["Lagged_tPLUS2"]

        df["lagret_1"]  = lr
        df["lagret_3"]  = lr.rolling(3,  min_periods=1).mean()
        df["lagret_5"]  = lr.rolling(5,  min_periods=1).mean()
        df["lagret_10"] = lr.rolling(10, min_periods=1).mean()

        df["lagvol_5"]  = lr.rolling(5,  min_periods=1).std()
        df["lagvol_10"] = lr.rolling(10, min_periods=1).std()

        df["lagabs_1"] = lr.abs()
        df["lagabs_5"] = lr.abs().rolling(5, min_periods=1).mean()

        roll_mean_20 = lr.rolling(20, min_periods=5).mean()
        roll_std_20  = lr.rolling(20, min_periods=5).std()
        df["lagz_20"] = (lr - roll_mean_20) / roll_std_20

        df["lagret1_x_vol5"]   = df["lagret_1"] * df["lagvol_5"]
        df["lagret1_x_z20"]    = df["lagret_1"] * df["lagz_20"]
        df["lagabs1_x_vol10"]  = df["lagabs_1"] * df["lagvol_10"]

        sign = np.sign(lr)
        df["lag_sign"]        = sign
        df["lag_sign_sum3"]   = sign.rolling(3, min_periods=1).sum()
        df["lag_sign_sum5"]   = sign.rolling(5, min_periods=1).sum()

    for col in etf_base_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[f"{col}_lag1"] = df[col].shift(1)
        df[f"{col}_lag2"] = df[col].shift(2)

        etf_ret = df[col].pct_change()
        df[f"{col}_ret1"]      = etf_ret.shift(1)
        df[f"{col}_ret5_mean"] = etf_ret.rolling(5, min_periods=3).mean().shift(1)
        df[f"{col}_ret5_std"]  = etf_ret.rolling(5, min_periods=3).std().shift(1)

    return df


# ============================================================
# Load Model + Top Features
# ============================================================

def load_model_and_features():
    artifact = joblib.load(MODEL_PKL_PATH)

    model = artifact.get("model", None)
    top_features = artifact.get("top_features", None)
    stored_cutoff = artifact.get("cutoff", PROBA_CUTOFF)

    if model is None:
        raise ValueError("Model artifact does not contain 'model'.")

    if top_features is None:
        feats_df = pd.read_csv(TOP_FEATURES_CSV_PATH)
        if "feature" in feats_df.columns:
            top_features = feats_df["feature"].tolist()
        else:
            top_features = feats_df.iloc[:, 0].tolist()

    print("Model trained up to date:", artifact.get("date_trained_upto", "N/A"))
    print("Top features loaded:", len(top_features))

    return model, top_features, stored_cutoff


# ============================================================
# Main Prediction Function
# ============================================================

def run_prediction():
    df = pd.read_csv(RAW_DATA_PATH)

    # Date parsing
    df[DATE_COL] = pd.to_datetime(df[DATE_COL], dayfirst=True, errors="coerce")
    df = df.sort_values(DATE_COL).reset_index(drop=True)

    # Identify CORE F1–F139 and ETF columns
    core_F_cols = sorted([
        c for c in df.columns
        if c.startswith("F") and c[1:].isdigit() and int(c[1:]) <= 139
    ])

    exclude_for_etf = set(core_F_cols + [DATE_COL, "Lagged_tPLUS2"])
    etf_base_cols = [c for c in df.columns if c not in exclude_for_etf]

    # Add technical features
    df = add_technical_features(df, core_F_cols, etf_base_cols)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Load model & features
    model, top_features, stored_cutoff = load_model_and_features()

    cutoff = PROBA_CUTOFF if PROBA_CUTOFF else stored_cutoff

    # Select top features
    X = df[top_features].copy()

    # Predict
    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(X)[:, 1]
    else:
        scores = model.decision_function(X)
        probs = 1.0 / (1.0 + np.exp(-scores))

    df["proba_gbm"] = probs
    df["signal"] = (df["proba_gbm"] >= cutoff).astype(int)

    # Extract last N rows only
    df_out = df.tail(N_PREDICT_LAST_ROWS).copy()
    df_out.to_csv(OUTPUT_PRED_PATH, index=False)

    print(f"\nSaved predictions (last {N_PREDICT_LAST_ROWS} rows) to:")
    print(" ", OUTPUT_PRED_PATH)
    print("\nPreview:")
    print(df_out[[DATE_COL, "proba_gbm", "signal"]])

    return df_out


# ============================================================
# Run
# ============================================================

results = run_prediction()


CWD: D:\work\Client\Morgan Stanley\Model Results


C:\Users\avina\AppData\Local\Temp\ipykernel_25340\3315020232.py:77: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  etf_ret = df[col].pct_change()
C:\Users\avina\AppData\Local\Temp\ipykernel_25340\3315020232.py:77: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  etf_ret = df[col].pct_change()
C:\Users\avina\AppData\Local\Temp\ipykernel_25340\3315020232.py:77: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
 

Model trained up to date: 2025-10-02
Top features loaded: 50

Saved predictions (last 2 rows) to:
  D:\work\Client\Morgan Stanley\Model Results\MSZZMOMO_GBM_18M_CORE_top50_latest_preds.csv

Preview:
           DATE  proba_gbm  signal
1462 2025-10-01   0.686950       1
1463 2025-10-02   0.520614       0
